# ESM の埋め込みベクトルを取得する

## 1. 必要なライブラリのインストール

In [2]:
!pip install fair-esm

In [1]:
import esm
import numpy as np
import os
import torch
from tqdm import tqdm
from Bio import SeqIO

In [2]:
# 一応環境変数を変更
os.environ["HF_HOME"] = "D:/hf-home"

# ESM-1b を読み込み
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

# デバイスを決定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 評価モードにする
model.eval()

ProteinBertModel(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (embed_positions): LearnedPositionalEmbedding(1026, 1280, padding_idx=1)


In [3]:
data_dir = "../data/gds_dataset"  # データセットのディレクトリ
filenames = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, file))]  # .txt のみ抽出

save_to = "../data/embedding-vectors/esm1b"
os.makedirs(save_to, exist_ok=True)  # 保存先ディレクトリの作成

In [6]:
exclusion_bases = list("BJOUXZ")

def check_valid_seq(seq):
    """
    有効なアミノ酸配列かどうかを返す関数
    :param seq: タンパク質配列
    :return: 有効なタンパク質ならば True
    """

    # 配列長が1000以上なら除外
    if len(seq) >= 1000:
        return False

    # "BJOUXZ" が含まれていたら除外
    for base in exclusion_bases:
        if base in seq:
            return False

    return True

### 1.1 データの作成

In [7]:
data = []

for filename in tqdm(filenames):
    with open(filename, "r", encoding="utf-8") as handle:
        for record in SeqIO.parse(filename, "fasta"):
            accession_number = record.id.split("|")[3] if len(record.id.split("|")[3]) > 0 else "Unknown"  # アクセッション番号を取得
            seq = record.seq

            # 有効なアミノ酸配列かを確認
            if check_valid_seq(seq):
                data.append((accession_number, seq))  # データに追加

100%|██████████| 87/87 [00:00<00:00, 516.36it/s]


## 2. ESM-1b で埋め込みベクトルを作成 して npy へ保存

In [10]:
batch_size = 2  # バッチサイズ

number = 1

for i in tqdm(range(0, len(data), batch_size)):
    batch = data[i:i + batch_size]
    batch_labels, batch_strs, batch_tokens = batch_converter(batch)
    batch_tokens = batch_tokens.to(device)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # 残基ごとの表現を抽出する
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)

    reps = results["representations"][33].to("cpu")

    for b in range(len(batch)):
        rep = reps[b, 1:len(batch_strs[b]) + 1, :].numpy()
        np.save(os.path.join(save_to, f"{number}.npy"), rep)
        number += 1

100%|██████████| 3858/3858 [19:12<00:00,  3.35it/s]
